In [1]:
elastic_rods_dir = '../../../elastic_rods/python/'
weaving_dir = '../../'
import os
import os.path as osp
import sys; sys.path.append(elastic_rods_dir); sys.path.append(weaving_dir)
import numpy as np, elastic_rods, linkage_vis
import numpy.linalg as la
from bending_validation import suppress_stdout as so
import matplotlib.pyplot as plt
from elastic_rods import EnergyType, InterleavingType

# weaving
import analysis_helper, ribbon_linkage_helper, mesh_vis, linkage_utils, compute_curve_from_curvature, pipeline_helper, importlib
importlib.reload(analysis_helper)
importlib.reload(ribbon_linkage_helper)
importlib.reload(mesh_vis)
importlib.reload(linkage_utils)
importlib.reload(compute_curve_from_curvature)
importlib.reload(pipeline_helper)
from analysis_helper import (compare_turning_angle,
                            is_on_sphere, 
                            get_distance_to_center_scalar_field, 
                            plot_curvatures, 
                            get_curvature_scalar_field,
                            construct_elastic_rod_loop_from_rod_segments, 
                            concatenate_rod_properties_from_rod_segments, 
                            compute_min_distance_rigid_transformation)
from ribbon_linkage_helper import (update_rest_curvature, 
                                   set_ribbon_linkage,
                                   export_linkage_geometry_to_obj,
                                   write_linkage_ribbon_output_florin)
from compute_curve_from_curvature import (match_geo_curvature_and_edge_len, get_all_curve_pattern)
from linkage_utils import order_segments_by_ribbons, get_turning_angle_and_length_from_ordered_rods

from pipeline_helper import (initialize_linkage, get_normal_deviation, set_joint_vector_field, stage_1_optimization, initialize_stage_2_optimizer, stage_2_optimization, InputOrganizer, write_all_output, set_surface_view_options, get_structure_analysis_view, get_double_side_view, show_selected_joints)
import vis.fields
import matplotlib.cm as cm
import time

In [2]:
# rod_length = curved_linkage.totalRestLength() / curved_linkage.numSegments()
rod_length = 0.11031757615154746
width = rod_length / 15 * 5
thickness = width / 5 * 0.5
print(width, thickness)
scale = 1
io = InputOrganizer('costa_{}'.format(scale), thickness, width, weaving_dir)

0.03677252538384915 0.0036772525383849146


In [3]:
OPTS = elastic_rods.NewtonOptimizerOptions()
OPTS.gradTol = 1e-6
OPTS.verbose = 1;
OPTS.beta = 1e-8
OPTS.niter = 200
OPTS.verboseNonPosDef = False
rw = 0.01
sw = 0.01

In [4]:
with so(): curved_linkage = initialize_linkage(surface_path = io.SURFACE_PATH, useCenterline = True, model_path = io.MODEL_PATH, cross_section = io.RIBBON_CS, subdivision_res = io.SUBDIVISION_RESOLUTION)
curved_linkage.set_design_parameter_config(use_restLen = True, use_restKappa = True)
curved_save_tgt_joint_pos = curved_linkage.jointPositions();
curved_linkage_view = get_double_side_view(curved_linkage, flip = True)
curved_linkage_view.show()

Renderer(camera=PerspectiveCamera(aspect=1.6, children=(PointLight(color='white', intensity=0.6, position=(0.0…

In [5]:
curved_linkage.setExtendedDoFsPSRL(np.load('costa_1_dof.npy'))

In [6]:
bottomColor =[180/255., 158/255., 119/255.]
topColor =[79/255., 158/255., 246/255.]
heights = curved_linkage.visualizationGeometryHeightColors()
colors = np.take(np.array([bottomColor, topColor]), heights < heights.mean(), axis=0)

In [7]:
curved_linkage.attraction_weight = 1e-5

In [8]:
def eqm_callback(prob, i):
    curved_linkage_view.update(scalarField = colors)

In [9]:
elastic_rods.compute_equilibrium(curved_linkage, callback = eqm_callback, options = OPTS)

0	0.02872	6.77387	6.77387	1	1
1	0.00178561	1.25605	1.25605	1	1
2	0.000511713	0.204562	0.204562	1	1
3	0.000307892	0.042304	0.042304	1	1
4	0.000222923	0.0159791	0.0159791	1	1
5	0.000172387	0.0074696	0.0074696	1	1
6	0.00014325	0.00386303	0.00386303	1	1
7	0.000128449	0.00211651	0.00211651	1	0
8	0.000118502	0.00915285	0.00915285	1	0
9	0.00011846	7.07083e-06	7.07083e-06	1	0
10	0.00011846	7.07103e-08	7.07103e-08	1	0


In [10]:
def visualize_crossing(index, linkage, view):
    vf = show_selected_joints(linkage, [index], flip = True)
    view.update(vectorField = vf, scalarField = colors)

In [11]:
ribbons = order_segments_by_ribbons(curved_linkage)
_, _, _, _, all_joint_index, _ = get_turning_angle_and_length_from_ordered_rods(ribbons, curved_linkage, rest = True)
all_joint_index_list = [j_list + [j_list[0]] for j_list in all_joint_index]

def get_ribbon_crossing_list(index, linkage, view):
    selected_list = []
    selected_ribbon = []
    for ribbon_index, index_list in enumerate(all_joint_index_list):
        if index in set(index_list):
            selected_ribbon.append(ribbon_index)
            selected_list.append(index_list)
    print("The crossing {} belongs to ribbon {}".format(index, ', '.join([str(x) for x in selected_ribbon])))
    for i in range(len(selected_list)):
        print('Ribbon {}: {}'.format(selected_ribbon[i], selected_list[i]))

In [61]:
def select_crossing(crossing_index):
    visualize_crossing(crossing_index, curved_linkage, curved_linkage_view)
    get_ribbon_crossing_list(crossing_index, curved_linkage, curved_linkage_view)
select_crossing(199)

The crossing 199 belongs to ribbon 13
Ribbon 13: [154, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 102, 211, 53, 212, 213, 214, 198, 215, 216, 151, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 214, 231, 232, 50, 191, 185, 233, 234, 139, 235, 236, 237, 238, 239, 240, 241, 242, 199, 230, 243, 244, 245, 246, 247, 248, 249, 6, 109, 65, 250, 251, 192, 154]


In [66]:
vf = show_selected_joints(curved_linkage, [242, 198, 199, 200, 230, 229, 214, 243])
curved_linkage_view.update(vectorField = vf, scalarField = colors)
curved_linkage_view.show()

Renderer(camera=PerspectiveCamera(aspect=1.6, children=(PointLight(color='white', intensity=0.6, position=(0.0…